In [1]:
from ozon_matching.kopatych_solution.utils import read_parquet, extract_category_levels
import polars as pl
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
import numpy as np

In [2]:
data = pl.concat(
    [
        read_parquet('../data/test_data.parquet', columns=['variantid', 'categories']),
        read_parquet('../data/train_data.parquet', columns=['variantid', 'categories']),
    ]
)
data = data.unique(subset=['variantid'])

2023-05-23 11:31:17.916 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_data.parquet
2023-05-23 11:31:17.927 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 35730, N Cols - 2
2023-05-23 11:31:17.928 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_data.parquet
2023-05-23 11:31:18.047 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 457063, N Cols - 2


In [3]:
data = extract_category_levels(data, [3, 4])
data = data.select(pl.col(['variantid', 'category_level_3', 'category_level_4']))

In [4]:
data = data.join(
    (
        data
        .select(pl.col('category_level_3'))
        .unique()
        .with_row_count(name='category_level_3_id')
    ),
    on=['category_level_3']
)

data = data.join(
    (
        data
        .select(pl.col('category_level_4'))
        .unique()
        .with_row_count(name='category_level_4_id')
    ),
    on=['category_level_4']
)

data = data.drop(['category_level_3', 'category_level_4'])

In [5]:
train = read_parquet('../data/train_pairs.parquet')
train = train.drop(['target']).with_columns([pl.lit(1).cast(pl.Int8).alias('is_train')])

test = read_parquet('../data/test_pairs_wo_target.parquet')
test = test.drop(['__index_level_0__']).with_columns([pl.lit(0).cast(pl.Int8).alias('is_train')])

2023-05-23 11:31:18.912 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_pairs.parquet
2023-05-23 11:31:18.922 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 306540, N Cols - 3
2023-05-23 11:31:18.923 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_pairs_wo_target.parquet
2023-05-23 11:31:18.926 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 18084, N Cols - 3


In [6]:
pairs = pl.concat([train, test])

In [7]:
pairs = pairs.join(
    data.rename(
        {
            'variantid': 'variantid1',
            'category_level_3_id': 'category_level_3_id_1',
            'category_level_4_id': 'category_level_4_id_1',
        }
    ),
    on=['variantid1']
).join(
    data.rename(
        {
            'variantid': 'variantid2',
            'category_level_3_id': 'category_level_3_id_2',
            'category_level_4_id': 'category_level_4_id_2',
        }
    ),
    on=['variantid2']
).drop(['variantid1', 'variantid2']).to_pandas()

In [10]:

cv1 = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)
cv2 = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)

X = pairs.drop(columns=['is_train']).values
y = pairs['is_train'].values

scores_cv = []

for cv_index, holdout_index in tqdm(cv1.split(X, y)):
    X_cv, X_holdout = X[cv_index], X[holdout_index]
    y_cv, y_holdout = y[cv_index], y[holdout_index]
    for train_index, valid_index in tqdm(cv2.split(X_cv, y_cv)):
        X_train, X_valid = X_cv[train_index], X_cv[valid_index]
        y_train, y_valid = y_cv[train_index], y_cv[valid_index]

    
        model = LGBMClassifier(
            n_estimators=1000,
        )
        model.fit(
            X=X_train,
            y=y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric=['auc'],
            categorical_feature=[0,1,2,3],
            early_stopping_rounds=50
        )
        scores_cv.append(roc_auc_score(y_holdout, model.predict_proba(X_holdout)[:, 1]))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[1]	valid_0's auc: 0.78224	valid_0's binary_logloss: 0.207761
[2]	valid_0's auc: 0.784512	valid_0's binary_logloss: 0.202928
[3]	valid_0's auc: 0.785514	valid_0's binary_logloss: 0.199343
[4]	valid_0's auc: 0.785763	valid_0's binary_logloss: 0.196546
[5]	valid_0's auc: 0.785699	valid_0's binary_logloss: 0.194334
[6]	valid_0's auc: 0.785663	valid_0's binary_logloss: 0.192543
[7]	valid_0's auc: 0.785923	valid_0's binary_logloss: 0.191055
[8]	valid_0's auc: 0.786015	valid_0's binary_logloss: 0.189815
[9]	valid_0's auc: 0.786224	valid_0's binary_logloss: 0.188745
[10]	valid_0's auc: 0.786124	valid_0's binary_logloss: 0.18786
[11]	valid_0's auc: 0.786112	valid_0's binary_logloss: 0.187101
[12]	valid_0's auc: 0.786036	valid_0's binary_logloss: 0.18646
[13]	valid_0's auc: 0.786193	valid_0's binary_logloss: 0.185895
[14]	valid_0's auc: 0.786314	valid_0's binary_logloss: 0.185401
[15]	valid_0's auc: 0.786272	valid_0's binary_logloss: 0.184974
[16]	valid_0's auc: 0.786267	valid_0's binary_loglos

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

...

[1]	valid_0's auc: 0.779909	valid_0's binary_logloss: 0.207739
[2]	valid_0's auc: 0.781427	valid_0's binary_logloss: 0.20297
[3]	valid_0's auc: 0.781571	valid_0's binary_logloss: 0.199475
[4]	valid_0's auc: 0.78189	valid_0's binary_logloss: 0.196778
[5]	valid_0's auc: 0.782034	valid_0's binary_logloss: 0.19462
[6]	valid_0's auc: 0.782194	valid_0's binary_logloss: 0.192852
[7]	valid_0's auc: 0.782631	valid_0's binary_logloss: 0.191416
[8]	valid_0's auc: 0.782874	valid_0's binary_logloss: 0.190207
[9]	valid_0's auc: 0.782955	valid_0's binary_logloss: 0.18918
[10]	valid_0's auc: 0.783124	valid_0's binary_logloss: 0.188323
[11]	valid_0's auc: 0.783271	valid_0's binary_logloss: 0.187593
[12]	valid_0's auc: 0.78346	valid_0's binary_logloss: 0.186929
[13]	valid_0's auc: 0.78348	valid_0's binary_logloss: 0.186374
[14]	valid_0's auc: 0.783534	valid_0's binary_logloss: 0.185909
[15]	valid_0's auc: 0.783583	valid_0's binary_logloss: 0.185499
[16]	valid_0's auc: 0.783677	valid_0's binary_logloss: 

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[14]	valid_0's auc: 0.790437	valid_0's binary_logloss: 0.184748
[15]	valid_0's auc: 0.79056	valid_0's binary_logloss: 0.184322
[16]	valid_0's auc: 0.790621	valid_0's binary_logloss: 0.183935
[17]	valid_0's auc: 0.790733	valid_0's binary_logloss: 0.183609
[18]	valid_0's auc: 0.790655	valid_0's binary_logloss: 0.183326
[19]	valid_0's auc: 0.790593	valid_0's binary_logloss: 0.183068
[20]	valid_0's auc: 0.790746	valid_0's binary_logloss: 0.18285
[21]	valid_0's auc: 0.790709	valid_0's binary_logloss: 0.182656
[22]	valid_0's auc: 0.790685	valid_0's binary_logloss: 0.182491
[23]	valid_0's auc: 0.790647	valid_0's binary_logloss: 0.18234
[24]	valid_0's auc: 0.79064	valid_0's binary_logloss: 0.18221
[25]	valid_0's auc: 0.790696	valid_0's binary_logloss: 0.182099
[26]	valid_0's auc: 0.790681	valid_0's binary_logloss: 0.181997
[27]	valid_0's auc: 0.790503	valid_0's binary_logloss: 0.181915
[28]	valid_0's auc: 0.79045	valid_0's binary_logloss: 0.181844
[29]	valid_0's auc: 0.790311	valid_0's binary_

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[1]	valid_0's auc: 0.777747	valid_0's binary_logloss: 0.207712
[2]	valid_0's auc: 0.778499	valid_0's binary_logloss: 0.203008
[3]	valid_0's auc: 0.7789	valid_0's binary_logloss: 0.199572
[4]	valid_0's auc: 0.778905	valid_0's binary_logloss: 0.196929
[5]	valid_0's auc: 0.778633	valid_0's binary_logloss: 0.194854
[6]	valid_0's auc: 0.778653	valid_0's binary_logloss: 0.193145
[7]	valid_0's auc: 0.778516	valid_0's binary_logloss: 0.191761
[8]	valid_0's auc: 0.779283	valid_0's binary_logloss: 0.190586
[9]	valid_0's auc: 0.779543	valid_0's binary_logloss: 0.189611
[10]	valid_0's auc: 0.779576	valid_0's binary_logloss: 0.188787
[11]	valid_0's auc: 0.779718	valid_0's binary_logloss: 0.188095
[12]	valid_0's auc: 0.779651	valid_0's binary_logloss: 0.187483
[13]	valid_0's auc: 0.779767	valid_0's binary_logloss: 0.186954
[14]	valid_0's auc: 0.779805	valid_0's binary_logloss: 0.186512
[15]	valid_0's auc: 0.779715	valid_0's binary_logloss: 0.186117
[16]	valid_0's auc: 0.779776	valid_0's binary_loglo

In [11]:
scores_cv

[0.7876221624506864,
 0.7867872373773261,
 0.786519549789161,
 0.7797414551876096,
 0.7806722783473263,
 0.7792948011300385,
 0.7913433107929201,
 0.7907979311537467,
 0.7912293564211352,
 0.777790702181883,
 0.7783926835859292,
 0.7787539490909074,
 0.7825834469876747,
 0.7820520037807075,
 0.7832959901507028]

In [13]:
np.mean(scores_cv)

0.7837917905618502